In [16]:
!pip install numpy 
!pip install pandas
!pip install openpyxl
!pip install scikit-learn
!pip install matplotlib
!pip install Flask joblib
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 27.5 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import joblib
from joblib import dump 

In [27]:
file_path = 'Calvingdata_p4.xlsx'  
df = pd.read_excel(file_path)
 
df_cleaned = df.dropna().reset_index(drop=True)  

In [28]:
def calculate_tilt_angle(accel_x, accel_y, accel_z):
    # Formula for tilt angle from accelerometer data (in degrees)
    return np.arctan2(np.sqrt(accel_x**2 + accel_y**2), accel_z) * (180.0 / np.pi)

# Calculate tilt angle for each row
df_cleaned['tilt_angle'] = calculate_tilt_angle(df_cleaned['accel_x'], df_cleaned['accel_y'], df_cleaned['accel_z'])

# Drop rows with NaN values in important columns
df_cleaned = df_cleaned.dropna(subset=['tilt_angle', 'gyro_x', 'gyro_y', 'gyro_z'])

# Rule-based classification for generating labels
def classify_activity(row):
    # Lying: Tilt angle between 70-90 degrees, and gyroscope values close to zero
    if 70 <= row['tilt_angle'] <= 90 and np.abs(row[['gyro_x', 'gyro_y', 'gyro_z']]).max() < 0.5:
        return 'Lying'
    # Standing: Tilt angle less than 30 degrees, moderate gyroscope values
    elif row['tilt_angle'] < 30:
        return 'Standing'
    # Sitting: Tilt angle between 30-70 degrees
    else:
        return 'Sitting'

# Apply the classification to generate labels
df_cleaned['label'] = df_cleaned.apply(classify_activity, axis=1)

# Now define features and target
X = df_cleaned[['accel_x', 'accel_z', 'tilt_angle']]  
y = df_cleaned['label']  # Generated label

In [29]:
# Initialize a Random Forest Classifier
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
cv_scores = cross_val_score(clf, X, y, cv=5)
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean Cross-Validation Accuracy:", cv_scores.mean())

joblib.dump(clf, 'activity_classifier_model.pkl')

Accuracy: 0.9979859013091642
              precision    recall  f1-score   support

     Sitting       1.00      1.00      1.00       505
    Standing       1.00      1.00      1.00       488

    accuracy                           1.00       993
   macro avg       1.00      1.00      1.00       993
weighted avg       1.00      1.00      1.00       993

Cross-Validation Accuracy Scores: [1.    0.995 1.    1.    1.   ]
Mean Cross-Validation Accuracy: 0.999


['activity_classifier_model.pkl']